# 0. Extract from MXL

In [ ]:
import zipfile
import glob

filename = "XMLs/Alta_cancion.mxl"
with zipfile.ZipFile(filename, 'r') as zipfile:
    zipfile.extractall('XMLs/Alta_cancion')

# 1. Read XML

In [119]:
import xml.etree.cElementTree as ET

filename = "../../XMLs/至少還有你_主旋律.musicxml"
tree = ET.parse(filename)

In [120]:
root = tree.getroot()
root.tag, root.attrib

('score-partwise', {'version': '3.1'})

In [121]:
all_notes = root.findall('.//note')

In [122]:
lyric_notes = []
for note in all_notes:
    n = {}
    if note.find('lyric') is not None:
        n['type'] = note.find('type').text
        n['lyric'] = note.find('./lyric/text').text
        n['pitch'] = note.find('./pitch/step').text + note.find('./pitch/octave').text
        if note.find('./pitch/alter') is not None:
            if note.find('./pitch/alter').text == '-1':
                p = n['pitch'][0] + 'b' + n['pitch'][1]
                n['pitch'] = p
            else:
                n['pitch'] = n['pitch'][0] + '#' + n['pitch'][1]
                
        lyric_notes.append(n)

In [123]:
len(lyric_notes)

159

In [124]:
lyric_notes

[{'type': '16th', 'lyric': '我', 'pitch': 'G4'},
 {'type': '16th', 'lyric': '怕', 'pitch': 'G4'},
 {'type': 'eighth', 'lyric': '來', 'pitch': 'F#4'},
 {'type': '16th', 'lyric': '不', 'pitch': 'A4'},
 {'type': '16th', 'lyric': '幾', 'pitch': 'D5'},
 {'type': '16th', 'lyric': '我', 'pitch': 'A4'},
 {'type': '16th', 'lyric': '要', 'pitch': 'A4'},
 {'type': 'eighth', 'lyric': '抱', 'pitch': 'D5'},
 {'type': '16th', 'lyric': '著', 'pitch': 'A4'},
 {'type': '16th', 'lyric': '你', 'pitch': 'B4'},
 {'type': '16th', 'lyric': '直', 'pitch': 'G4'},
 {'type': '16th', 'lyric': '到', 'pitch': 'G4'},
 {'type': 'eighth', 'lyric': '感', 'pitch': 'F#4'},
 {'type': 'eighth', 'lyric': '覺', 'pitch': 'A4'},
 {'type': 'eighth', 'lyric': '你', 'pitch': 'D5'},
 {'type': '16th', 'lyric': '的', 'pitch': 'E5'},
 {'type': '16th', 'lyric': '皺', 'pitch': 'E5'},
 {'type': 'eighth', 'lyric': '紋', 'pitch': 'D5'},
 {'type': '16th', 'lyric': '有', 'pitch': 'D5'},
 {'type': '16th', 'lyric': '了', 'pitch': 'E5'},
 {'type': 'eighth', 'lyric

# 2. Exploring 

In [88]:
time = root.find('.//time')

In [90]:
time.find(".//beats").text

'4'

In [92]:
time.find(".//beat-type").text

'4'

# 3. Helper classes /functions

## 3.1 Helper classes

In [125]:
from enum import Enum

class NoteTypes(Enum):
    '''Class defining types of notes'''
    whole = 1
    half = 2
    quarter = 4
    eight = 8
    sixteenth = 16
    thirtysecond = 32

In [126]:
class NoteLetters(Enum):
    C = 0
    D = 2
    E = 4
    F = 6
    G = 8
    A = 10
    B = 12

In [127]:
n2 = Note('quarter','C','4',None,None)
n2.printNote()

Note: C4 / quarter / None


In [128]:
#raise by 2 semitones
newval = getattr(NoteLetters,n2.step).value+2
newval

2

In [129]:
newval%14

2

In [130]:
NoteLetters(newval%14)

<NoteLetters.D: 2>

In [131]:
NoteLetters(newval).name

'D'

In [132]:
#lower by 2 semitones
newval = getattr(NoteLetters,n2.step).value-2
newval

-2

In [133]:
NoteLetters(newval%14)

<NoteLetters.B: 12>

In [134]:
NoteLetters(newval%14).name

'B'

In [135]:
#new note at the nottom, lower by 2 semitones
n3 = Note('quarter','B','4',None,None)
n3.printNote()
newval = getattr(NoteLetters,n3.step).value+2
newval

Note: B4 / quarter / None


14

In [136]:
NoteLetters(newval%14)

<NoteLetters.C: 0>

In [137]:
NoteLetters(newval%14).name

'C'

In [242]:
class Note:
    '''Class representing a musical note'''
    notetype = None
    step = ''
    octave = 0
    so = None
    tie = None
    accidental = None
    
    def __init__(self,notetype,step,octave,tie,accidental):
        self.notetype = notetype
        self.step = step
        self.octave = int(octave)
        self.tie = tie
        self.accidental = accidental
        
    def get_so(self):
        if self.accidental:
            so = self.step + self.accidental + str(self.octave)
        else:
            
            so = self.step + str(self.octave)
        return so
    
    def printNote(self):
        if self.step=='':
            print('Note: ' + 'rest' + ' / ' +
              str(self.notetype))
        else:
            print('Note: ' + str(self.get_so()) + ' / ' +
              str(self.notetype) + ' / ' +
              str(self.tie))
            
    def addOctaves(self,addOct):
        self.octave += addOct
        
    def copyNote(self):
        ncopy = Note(self.notetype,self.step,self.octave,self.tie,self.accidental)
        return ncopy
    
    def addSemiTone(self,semiTToAdd):
        
        #get relative position of new note
        newVal = getattr(NoteLetters,self.step).value + semiTToAdd
        
        #handle semitones first
        if (semiTToAdd%2!=0):
            
            if self.accidental == '#':
                self.accidental = None
                newVal += 1

            elif self.accidental == 'b':
                self.accidental = None
                newVal -= 1
            
            elif not self.accidental:
                self.accidental = '#'
                newVal -= 1
                
            #natural, assumes sharps in score
            else:
                self.accidental = None

        

       
        #Change step within same octave
        if newVal>=0 and newVal<14:
            self.step=NoteLetters(newVal).name
        
        #change step to a lower octave
        elif newVal<0:
            self.step = NoteLetters(newVal%14).name
            self.octave -= 1
        
        #change step to a higher octave
        elif newVal>=14:
            self.step = NoteLetters(newVal%14).name
            self.octave += 1     

        

In [202]:
class Measure:
    '''Class representing a measure'''
    notes = []
    
    def __init__(self,notes):
        self.notes = notes
    
    def get_sos(self):
        sos = []
        for note in self.notes:
            sos.append(note.get_so())
        return sos
    
    def get_notetypes(self):
        notetypes = []
        for note in self.notes:
            notetypes.append(note.notetype)
        return notetypes 
    
    def get_ties(self):
        ties = []
        for tie in self.notes:
            ties.append(note.tie)
        return ties 
    
    def printMeasure(self):
        for note in self.notes:
            note.printNote()

## 3.2 Helper functions

In [203]:
def get_ts_xml(root):
    '''Gets time signature for a musicxml file.
    Input: Root of XML Tree
    Output:top and bottom numbers of the Time signature'''
    time = root.find('.//time')
    top = int(time.find(".//beats").text)
    bottom = int(time.find(".//beat-type").text)
    return top,bottom

In [204]:
get_ts_xml(root)

(4, 4)

In [205]:
def get_bpm_xml(root):
    '''Gets beats per minute as well as type of beats for a musicxml file.
    Input: Root of XML Tree
    Output: beats per minute, noteType'''
    metronome = root.find('.//metronome')
    beats_per_min = int(metronome.find('.//per-minute').text)
    beat_type = metronome.find('.//beat-unit').text
    #beat_type = getattr(NoteTypes, str(beat_type))
    return beats_per_min, beat_type

In [206]:
get_bpm_xml(root)

(62, 'quarter')

In [207]:
def decode_xml_note(xml_note):
    '''Gets note type, step, octave and tie type'''
    try:
        notetype = xml_note.find('.//type').text
    except:
        notetype = None
    try:
        step = xml_note.find('.//step').text
    except:
        step = ''
    try:
        octave = xml_note.find('.//octave').text
    except:
        octave = 0
    try:
        tie = xml_note.find('.//tie').get('type')
    except:
        tie = None
    try:
        accidental = xml_note.find('.//accidental').text
        if accidental == 'natural':
            accidental = '*'
        if accidental == 'flat':
            accidental = 'b'
        if accidental == 'sharp':
            accidental = '#'
            
    except:
        accidental = None
        
    note = Note(notetype,step,octave,tie,accidental)
    #DEBUG
    #note.printNote()
    
    return note

In [208]:
measures = root.findall('.//measure')
xml_measure_notes = measures[4].findall('.//note')
xml_note = xml_measure_notes[0]
decode_xml_note(xml_note)

In [209]:
# return all notes in a measure
def get_meas_notes_xml(xml_measure_notes):
    notes = []
    for xml_note in xml_measure_notes:
        note = decode_xml_note(xml_note)
        if note.notetype:
            notes.append(note)
        #DEBUG
        #note.printNote()
    return notes

In [210]:
get_meas_notes_xml(xml_measure_notes)

## 3.3 Try to use classes

In [211]:
#get a note
xml_measures = root.findall('.//measure')
xml_measure = xml_measures[3].findall('.//note')
xml_note = xml_measure[0]
note = decode_xml_note(xml_note)
note.printNote()

Note: rest / half


In [212]:
note.so

In [213]:
note.notetype

'half'

In [214]:
note.tie

In [215]:
#get a measure
measure = Measure(get_meas_notes_xml(xml_measure))
measure.printMeasure()

Note: rest / half
Note: rest / quarter
Note: rest / eighth
Note: G4 / 16th / None
Note: G4 / 16th / None


In [216]:
measure.get_sos()

['0', '0', '0', 'G4', 'G4']

In [217]:
measure.get_notetypes()

['half', 'quarter', 'eighth', '16th', '16th']

In [218]:
measure.get_ties()

[None, None, None, None, None]

In [219]:
#read all measures
xml_measures = root.findall('.//measure')
measures = []

for xml_measure in xml_measures:
    measure = Measure(get_meas_notes_xml(xml_measure))
    if not measure.get_sos()==['0']:
        measures.append(measure)

In [243]:
#try adding semitones
n1 = measures[1].notes[1]
n1.printNote()
n2 = n1.copyNote()
n2.printNote()

Note: A4 / 16th / None
Note: A4 / 16th / None


In [244]:
n2.addSemiTone(4)
n2.printNote()

Note: C5 / 16th / None


In [245]:
n2.addSemiTone(1)
n2.printNote()

Note: C#5 / 16th / None


In [246]:
n2.addSemiTone(-1)
n2.printNote()

Note: C5 / 16th / None


In [247]:
n2.addSemiTone(-4)
n2.printNote()

Note: A4 / 16th / None


# 4. Read XML and get measures to invert chords

In [479]:
xml_measures = root.findall('.//measure')
measures = []

for xml_measure in xml_measures:
    measure = Measure(get_meas_notes_xml(xml_measure))
    measures.append(measure)
    

In [474]:
for measure in measures:
    measure.printMeasure()